In [23]:
pip install requests beautifulsoup4 pandas tqdm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin


In [25]:
BASE = "https://www.shl.com"
CATALOG_URL = BASE + "/products/product-catalog/"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def fetch_page(start: int | None, type_: int = 1) -> str:
    """Download one catalog page."""
    params = {"type": type_}
    if start is not None:
        params["start"] = start
    print(f"Scraping: {CATALOG_URL}", params)
    res = requests.get(CATALOG_URL, params=params, headers=headers, timeout=20)
    res.raise_for_status()
    return res.text


def parse_individual_table(html: str) -> list[dict]:
    """Parse Individual Test Solutions table from one page."""
    soup = BeautifulSoup(html, "html.parser")

    # find the table whose first header cell says "Individual Test Solutions"
    target_table = None
    for table in soup.find_all("table"):
        first_th = table.find("th")
        if first_th and "Individual Test Solutions" in first_th.get_text(strip=True):
            target_table = table
            break

    if target_table is None:
        print("⚠️  No Individual Test Solutions table found on this page.")
        return []

    rows_data = []

    # each row has data-entity-id attribute
    for tr in target_table.find_all("tr", attrs={"data-entity-id": True}):
        title_td = tr.find("td", class_="custom__table-heading__title")
        a_tag = title_td.find("a")
        name = a_tag.get_text(strip=True)
        rel_url = a_tag["href"]
        full_url = urljoin(BASE, rel_url)

        tds = tr.find_all("td")

        # remote testing & adaptive/IRT flags
        remote_testing = bool(tds[1].find("span", class_="-yes")) if len(tds) > 1 else None
        adaptive_irt   = bool(tds[2].find("span", class_="-yes")) if len(tds) > 2 else None

        # test type letters (A, B, C, K, P, S, ...)
        if len(tds) > 3:
            test_type_letters = [
                span.get_text(strip=True)
                for span in tds[3].find_all("span", class_="product-catalogue__key")
            ]
        else:
            test_type_letters = []

        rows_data.append({
            "name": name,
            "url": full_url,
            "remote_testing": remote_testing,
            "adaptive_irt": adaptive_irt,
            "test_type_letters": test_type_letters,
        })

    return rows_data


all_products = []
start = 0

while True:
    html = fetch_page(start=start, type_=1)   # 🔴 NOTE: type_=1 for Individual Test Solutions
    page_rows = parse_individual_table(html)
    print(f"Found {len(page_rows)} products on this page\n")

    if not page_rows:
        break

    all_products.extend(page_rows)
    start += 12        # SHL site shows 12 rows per page

len(all_products)

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 0}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 12}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 24}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 36}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 48}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 60}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 72}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 84}
Found 12 products on this page

Scraping: https://www.shl.com/products/product-catalog/ {'type': 1, 'start': 96}
Found 12 product

384

In [26]:
df = pd.DataFrame(all_products)
print(len(df))
df.head()

384


,name,url,remote_testing,adaptive_irt,test_type_letters
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,True,False,"[A, E, B, C, D, P]"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,True,True,[K]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]


In [27]:
df.head()

,name,url,remote_testing,adaptive_irt,test_type_letters
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,True,False,"[A, E, B, C, D, P]"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,True,True,[K]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,True,False,[K]


In [28]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [29]:

details = []

for i, row in df.iterrows():
    url = row['url']
    print(f"Scraping details {i+1}/{len(df)}: {url}")
    
    try:
        resp = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(resp.text, "html.parser")
        
        # Description block
        desc_block = soup.find("div", class_="product__description")
        description = desc_block.get_text(" ", strip=True) if desc_block else ""
        
        # Duration
        dur_block = soup.find("div", class_="product__duration")
        duration = dur_block.get_text(" ", strip=True) if dur_block else ""
        
        details.append({
            "name": row['name'],
            "url": url,
            "description": description,
            "duration": duration,
            "remote_testing": row['remote_testing'],
            "adaptive_irt": row['adaptive_irt'],
            "test_type_letters": row['test_type_letters']
        })
        
        time.sleep(1)  # be nice to server
    
    except Exception as e:
        print("Error:", e)
        pass

details_df = pd.DataFrame(details)
details_df.to_csv("shl_catalog_full_details.csv", index=False)

details_df.head()


Scraping details 1/384: https://www.shl.com/products/product-catalog/view/global-skills-development-report/
Scraping details 2/384: https://www.shl.com/products/product-catalog/view/net-framework-4-5/
Scraping details 3/384: https://www.shl.com/products/product-catalog/view/net-mvc-new/
Scraping details 4/384: https://www.shl.com/products/product-catalog/view/net-mvvm-new/
Scraping details 5/384: https://www.shl.com/products/product-catalog/view/net-wcf-new/
Scraping details 6/384: https://www.shl.com/products/product-catalog/view/net-wpf-new/
Scraping details 7/384: https://www.shl.com/products/product-catalog/view/net-xaml-new/
Scraping details 8/384: https://www.shl.com/products/product-catalog/view/accounts-payable-new/
Scraping details 9/384: https://www.shl.com/products/product-catalog/view/accounts-payable-simulation-new/
Scraping details 10/384: https://www.shl.com/products/product-catalog/view/accounts-receivable-new/
Scraping details 11/384: https://www.shl.com/products/produ

,name,url,description,duration,remote_testing,adaptive_irt,test_type_letters
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,,,True,False,"[A, E, B, C, D, P]"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,,,True,True,[K]
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,,,True,False,[K]
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,,,True,False,[K]
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,,,True,False,[K]


In [30]:
len(details_df)

384

In [31]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

BASE = "https://www.shl.com"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

def scrape_individual_tests():
    all_rows = []

    # adjust range exactly as you used when you got 408 items
    for start in range(0, 372 + 12, 12):
        url = f"{BASE}/products/product-catalog/?start={start}&type=1"
        print("Scraping:", url)
        res = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(res.text, "html.parser")

        # find the table with heading "Individual Test Solutions"
        tables = soup.find_all("table")
        individual_table = None
        for table in tables:
            th = table.find("th")
            if th and "Individual Test Solutions" in th.get_text(strip=True):
                individual_table = table
                break

        if individual_table is None:
            print("⚠️ No Individual Test table found on:", url)
            continue

        rows = individual_table.find_all("tr", attrs={"data-entity-id": True})
        for row in rows:
            title_td = row.find("td", class_="custom__table-heading__title")
            link = title_td.find("a")

            name = link.get_text(strip=True)
            rel_url = link["href"]
            full_url = BASE + rel_url

            tds = row.find_all("td", class_="custom__table-heading__general")
            remote_td = tds[0] if len(tds) > 0 else None
            adaptive_td = tds[1] if len(tds) > 1 else None
            testtype_td = tds[2] if len(tds) > 2 else None

            remote = bool(remote_td and remote_td.find("span", class_="-yes"))
            adaptive = bool(adaptive_td and adaptive_td.find("span", class_="-yes"))

            test_types = []
            if testtype_td:
                for span in testtype_td.find_all("span", class_="product-catalogue__key"):
                    test_types.append(span.get_text(strip=True))

            all_rows.append({
                "name": name,
                "url": full_url,
                "remote_testing": remote,
                "adaptive": adaptive,
                "test_types": ",".join(test_types)
            })

    df = pd.DataFrame(all_rows)
    return df

df = scrape_individual_tests()
len(df), df.head()


Scraping: https://www.shl.com/products/product-catalog/?start=0&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=12&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=24&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=36&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=48&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=60&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=72&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=84&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=96&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=108&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=120&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=132&type=1
Scraping: https://www.shl.com/products/product-catalog/?start=144&type=1
Scraping: https://www.shl.com/products/product-catalog/?start

(377,
                                name  \
 0  Global Skills Development Report   
 1                .NET Framework 4.5   
 2                    .NET MVC (New)   
 3                   .NET MVVM (New)   
 4                    .NET WCF (New)   
 
                                                  url  remote_testing  \
 0  https://www.shl.com/products/product-catalog/v...            True   
 1  https://www.shl.com/products/product-catalog/v...            True   
 2  https://www.shl.com/products/product-catalog/v...            True   
 3  https://www.shl.com/products/product-catalog/v...            True   
 4  https://www.shl.com/products/product-catalog/v...            True   
 
    adaptive   test_types  
 0     False  A,E,B,C,D,P  
 1      True            K  
 2     False            K  
 3     False            K  
 4     False            K  )

In [32]:
# drop duplicates just in case
df = df.drop_duplicates(subset=["url"]).reset_index(drop=True)

# add a simple numeric id
df["id"] = df.index

# create a combined text field for embeddings later
def build_all_text(row):
    parts = [
        row["name"],
        f"Test types: {row['test_types']}" if row["test_types"] else "",
        "Remote testing available" if row["remote_testing"] else "",
        "Adaptive / IRT" if row["adaptive"] else "",
    ]
    return " | ".join([p for p in parts if p])

df["all_text"] = df.apply(build_all_text, axis=1)

df.head()
len(df)


185

In [33]:
df.to_csv("shl_individual_tests_catalog.csv", index=False)


Step 3 - Load the given Excel dataset

In [34]:
pip install openpyxl


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [35]:
import pandas as pd

xls_path = "Gen_AI Dataset.xlsx"  # adjust if in subfolder
xls = pd.ExcelFile(xls_path)
xls.sheet_names


['Train-Set', 'Test-Set']

In [36]:
train_df = pd.read_excel(xls, "Train-Set")   # if such sheet exists
test_df  = pd.read_excel(xls, "Test-Set")    # adjust names based on sheet_names
train_df.head(), test_df.head()


(                                               Query  \
 0  I am hiring for Java developers who can also c...   
 1  I am hiring for Java developers who can also c...   
 2  I am hiring for Java developers who can also c...   
 3  I am hiring for Java developers who can also c...   
 4  I am hiring for Java developers who can also c...   
 
                                       Assessment_url  
 0  https://www.shl.com/solutions/products/product...  
 1  https://www.shl.com/solutions/products/product...  
 2  https://www.shl.com/solutions/products/product...  
 3  https://www.shl.com/solutions/products/product...  
 4  https://www.shl.com/products/product-catalog/v...  ,
                                                Query
 0  Looking to hire mid-level professionals who ar...
 1  Job Description\n\n Join a community that is s...
 2  I am hiring for an analyst and wants applicati...
 3  I have a JD Job Description\n\n People Science...
 4  I am new looking for new graduates in my sale

In [37]:
# jo columns assessment URLs wale honge unko pick karo
url_cols = [c for c in train_df.columns if "Assessment" in c or "assessment" in c]

def normalize_url_safe(u):
    if pd.isna(u):
        return None
    return normalize_url(u)

for col in url_cols:
    train_df[col] = train_df[col].apply(normalize_url_safe)


In [38]:
import pandas as pd

def normalize_url(u):
    if pd.isna(u):
        return ""
    u = str(u)
    if "/view/" in u:
        u = u.split("/view/")[1]
    u = u.split("?")[0].split("#")[0].strip("/").lower()
    return u

df["key"] = df["url"].apply(normalize_url)
train_df["key"] = train_df["Assessment_url"].apply(normalize_url)
# test_df["key"]  = test_df["Assessment_url"].apply(normalize_url)


Next step

In [39]:
df = pd.read_csv("shl_catalog_full_details.csv")
df.head()


,name,url,description,duration,remote_testing,adaptive_irt,test_type_letters
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,NaN,NaN,True,False,"['A', 'E', 'B', 'C', 'D', 'P']"
1,.NET Framework 4.5,https://www.shl.com/products/product-catalog/v...,NaN,NaN,True,True,['K']
2,.NET MVC (New),https://www.shl.com/products/product-catalog/v...,NaN,NaN,True,False,['K']
3,.NET MVVM (New),https://www.shl.com/products/product-catalog/v...,NaN,NaN,True,False,['K']
4,.NET WCF (New),https://www.shl.com/products/product-catalog/v...,NaN,NaN,True,False,['K']


In [40]:
BASE_URL = "https://www.shl.com"

def normalize_url(u: str):
    if pd.isna(u):
        return None
    u = str(u).strip()
    # agar relative path hai to base add karo
    if u.startswith("/"):
        u = BASE_URL.rstrip("/") + u
    # extra trailing slash hata do
    if u.endswith("/"):
        u = u[:-1]
    # lower for safe comparison
    return u.lower()

# catalog df me normalized column banao
df["url_norm"] = df["url"].apply(normalize_url)


In [41]:
df.shape, df.columns


((384, 8),
 Index(['name', 'url', 'description', 'duration', 'remote_testing',
        'adaptive_irt', 'test_type_letters', 'url_norm'],
       dtype='object'))

In [42]:
# Basic text parts
name_col        = "name"        if "name"        in df.columns else df.columns[0]
desc_col        = "description" if "description" in df.columns else None
test_type_col   = "test_type"   if "test_type"   in df.columns else None

parts = [df[name_col].fillna("")]

if desc_col:
    parts.append(df[desc_col].fillna(""))

if test_type_col:
    parts.append(df[test_type_col].fillna(""))

# join pieces into one text column
df["text"] = (" . ".join(["{}"] * len(parts))).format(*parts)

df[["name" if "name" in df.columns else name_col, "text"]].head()


,name,text
0,Global Skills Development Report,0 Global Skills Development Report\...
1,.NET Framework 4.5,0 Global Skills Development Report\...
2,.NET MVC (New),0 Global Skills Development Report\...
3,.NET MVVM (New),0 Global Skills Development Report\...
4,.NET WCF (New),0 Global Skills Development Report\...


In [43]:
!pip install -q sentence-transformers scikit-learn



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
!pip install tf-keras==2.15 --quiet
!pip install "transformers==4.41.2" --quiet
!pip install sentence-transformers==2.7.0 --quiet


[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [46]:
import torch
from sentence_transformers import SentenceTransformer, util

# 1. Set device to CPU (simple and safe)
device = torch.device("cpu")

model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name, device=str(device))

# 2. Make sure df is loaded
# df = pd.read_csv("shl_catalog_full_details.csv")

# 3. Build combined text for each assessment
df["combined_text"] = (
    df["name"].fillna("") + " | " +
    df["description"].fillna("")  # or whatever column you used earlier
)

# 4. Compute catalog embeddings on CPU
catalog_embeddings = model.encode(
    df["combined_text"].tolist(),
    convert_to_tensor=True,
    device=str(device),
    show_progress_bar=True
)

print(catalog_embeddings.shape)


c:\Users\abhis\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\utils\_validators.py:186: UserWarning: The `resume_download` argument is deprecated and ignored in `hf_hub_download`. Downloads always resume whenever possible.
  `use_auth_token` is passed to a function, the `use_auth_token` value is passed
Batches: 100%|██████████| 12/12 [00:03<00:00,  3.11it/s]

torch.Size([384, 384])


In [47]:
# guess likely column names
from sentence_transformers import util

name_col = "name" if "name" in df.columns else df.columns[0]
url_col  = "url"  if "url"  in df.columns else "link"  # change if needed
test_type_col = "test_type" if "test_type" in df.columns else None

def recommend_assessments(query, top_k=5):
    # Encode query on the SAME device as catalog_embeddings
    q_emb = model.encode(
        [query],
        convert_to_tensor=True,
        device=str(device)
    )

    # Cosine similarity (both tensors on CPU)
    scores = util.cos_sim(q_emb, catalog_embeddings)[0].cpu().numpy()

    df["score"] = scores
    top = df.sort_values("score", ascending=False).head(top_k).copy()

    return top[["name", "url", "score"]]


In [48]:
sample_query = train_df["Query"].iloc[0]
sample_query


'I am hiring for Java developers who can also collaborate effectively with my business teams. Looking for an assessment(s) that can be completed in 40 minutes.'

In [49]:
recommend_assessments(sample_query, top_k=5)

,name,url,score
337,Software Business Analysis,https://www.shl.com/products/product-catalog/v...,0.463705
325,Software Business Analysis,https://www.shl.com/products/product-catalog/v...,0.463705
313,Software Business Analysis,https://www.shl.com/products/product-catalog/v...,0.463705
32,Assessment and Development Center Exercises,https://www.shl.com/products/product-catalog/v...,0.435966
0,Global Skills Development Report,https://www.shl.com/products/product-catalog/v...,0.409530


NEXT STEPS - Build the RAG / Retrieval Pipeline

In [50]:
def normalize_url(u: str) -> str:
    if not isinstance(u, str):
        return ""
    u = u.strip()
    u = u.replace("http://", "https://")
    # remove trailing slash for consistency
    if u.endswith("/"):
        u = u[:-1]
    return u

In [51]:
def evaluate_recall_at_k(train_df, k=10):
    recalls = []

    for q in train_df["Query"].unique():
        # all ground-truth URLs for this query
        true_urls = (
            train_df.loc[train_df["Query"] == q, "Assessment_url"]
            .dropna()
            .map(normalize_url)
            .tolist()
        )
        true_urls = set(true_urls)

        if not true_urls:
            continue

        preds = recommend_assessments(q, top_k=k)
        pred_urls = preds["url"].map(normalize_url).tolist()
        pred_urls = set(pred_urls)

        hits = len(true_urls & pred_urls)
        recalls.append(hits / len(true_urls))

    if not recalls:
        return 0.0

    return sum(recalls) / len(recalls)


In [52]:
for k in [1, 5, 10]:
    r = evaluate_recall_at_k(train_df, k=k)
    print(f"Recall@{k} = {r:.4f}")


Recall@1 = 0.0000


Recall@5 = 0.0000
Recall@10 = 0.0000


In [53]:
def gradio_recommend(query):
    if not query or not query.strip():
        return "Please enter a query.", []
    
    top = recommend_assessments(query, top_k=10)
    # convert to something Gradio-friendly, e.g. list of lists
    return top.to_dict(orient="records")


Step 4 — Build the Final Gradio App (Web-based RAG Tool)

In [54]:
import gradio as gr
import pandas as pd

def format_results(df):
    """Convert predictions to clickable table."""
    rows = []
    for _, row in df.iterrows():
        rows.append([
            row["name"],
            f"<a href='{row['url']}' target='_blank'>{row['url']}</a>",
            float(row["score"])
        ])
    return pd.DataFrame(rows, columns=["Assessment Name", "URL", "Score"])

def gradio_recommend(query, top_k):
    if not query.strip():
        return pd.DataFrame([["No input", "", ""]], columns=["Assessment Name", "URL", "Score"])

    results = recommend_assessments(query, top_k=top_k)
    return format_results(results)

demo = gr.Interface(
    fn=gradio_recommend,
    inputs=[
        gr.Textbox(lines=3, label="Enter role / JD / requirement"),
        gr.Slider(1, 10, value=5, step=1, label="Number of recommendations")
    ],
    outputs=gr.Dataframe(label="Recommended Assessments"),
    title="SHL Assessment Recommendation Engine",
    description="A RAG-based recommender system built for SHL's product catalog."
)

demo.launch(debug=True)


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
